# Atelier 5, PageRank et PageRank personnalisé #
Cet atelier porte sur l'algorithme PageRank et sa version personnalisée personnalisé pour effectuer des recommandations.  Utilisez la bibliothèque "igraph" pour cet exercice et les données de MovieLens 100k.

Chaque question compte pour 5 points.

## Question 1 ##
(1) Créez la matrice d'adjacence du graphe biparti que nous nommerons B.  Cette matrice doit avoir une dimension de (943+1682,943+1682).  Vous pouvez la créer à partir de la matrice d'adjacence des votes de dimension (943,1682).

In [42]:
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
import igraph as ig
import matplotlib.pyplot as plt

u_data = pd.read_csv("data/votes.csv")
u_user = pd.read_csv("data/u.csv")
u_item = pd.read_csv("data/items.csv")

# Matrice d'adjacence
m = u_data.pivot(index="user.id", columns="item.id", values="rating").to_numpy(na_value=0)
B = np.zeros((m.shape[0]+m.shape[1], m.shape[0]+ m.shape[1]))
B[:m.shape[0], :m.shape[1]] = m
B[m.shape[0]:, m.shape[1]:] = m.T
G = ig.Graph.Adjacency(B)
print(f"Matrice d'adjacence bipartie: {B.shape}")
B

Matrice d'adjacence bipartie: (2625, 2625)


array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Question 2 ##
(2) Calculez le vecteur du PageRank de la matrice biparti B.

In [45]:
r = G.pagerank()
r = np.array(r)
print(f"PageRank: {r.shape}, {r}")

PageRank: (2625,), [1.45008065e-03 1.85749925e-04 1.66346766e-04 ... 8.69887378e-05
 2.01711568e-04 9.61742966e-04]


## Question 3 ##
(3) Les titres ci-dessous représentent les films de deux réalisateurs, Alfred Hitchcock et Stanley Kubrick, que l'on retrouve dans les données Movielens.  Utilisez la version personnalisée de l'algorithme PageRank et présumez un utilisateur qui aime les deux premiers de chaque liste (Birds, Vertigo, Full Metal [...] et Clockwork [...]), mais qui n'a pas de vote pour les autres.  Quels sont les 80 premier films qui seraient recommandés et comment cela se compare-t-il avec la version non personnalisée?

In [78]:
hitchcock_titres = [
    "Birds, The (1963)",
    "Vertigo (1958)",
    "North by Northwest (1959)",
    "Rear Window (1954)",
    "Dial M for Murder (1954)",
    "Psycho (1960)",
]

kubrick_titres = [
    "Full Metal Jacket (1987)",
    "Clockwork Orange, A (1971)",
    "2001: A Space Odyssey (1968)",
    "Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)",
]

# personalisation est 1 si il a vu et 0 autrement
hitchcock_ids = u_item[u_item[' movie title '].isin(hitchcock_titres)]['movie id ']
kubrick_ids = u_item[u_item[" movie title "].isin(kubrick_titres)]["movie id "]

# Initialize personalization vector with zeros
reset = np.zeros(G.vcount())

# Set the value to 1 if the user has rated a Hitchcock or Kubrick movie
for user_id in range(1, len(u_user) + 1):
    user_ratings = u_data[u_data['user.id'] == user_id]
    if any(user_ratings['item.id'].isin(hitchcock_ids)) or any(user_ratings['item.id'].isin(kubrick_ids)):
        reset[user_id - 1] = 1

# Compute personalized PageRank
pr = G.personalized_pagerank(reset=reset)
pr = np.array(pr)

def show_top_80(pr):
    # show top  80
    pr = pr[:m.shape[0]]
    top_80_ids = np.argsort(pr)[-80:]
    recommendation = items.iloc[top_80_ids][" movie title "]
    return recommendation.reset_index(drop=True)


personalize, normal = show_top_80(pr), show_top_80(r)
df = pd.DataFrame({"Personalize": personalize, "Normal": normal})
with pd.option_context("display.max_rows", 80, "display.max_columns", 2, "display.width", 1000):
    print(df)

                                     Personalize                                        Normal
0                                 Twister (1996)                                Twister (1996)
1                             Stand by Me (1986)                                 Aliens (1986)
2                   2001: A Space Odyssey (1968)                                 Mother (1996)
3                             Ulee's Gold (1997)             E.T. the Extra-Terrestrial (1982)
4                                  Aliens (1986)                              Boot, Das (1981)
5                               Apollo 13 (1995)                        Terminator, The (1984)
6                            Forrest Gump (1994)                           Forrest Gump (1994)
7                                  Psycho (1960)                                Amadeus (1984)
8              E.T. the Extra-Terrestrial (1982)                When Harry Met Sally... (1989)
9                                 Amadeus (1984)  

Nous voyons qu'il y a des films similaires puisqu'ils sont populaires, mais nous voulons voir si l'algorithme PageRank peut nous donner des recommandations plus pertinentes. La version spécialisé donne en effet les films du même réalisateur haut dans les recommandations.

## Question 4 ##
(4) Comment pourrait-on intégrer à cet algorithme l'information du réalisateur de ces films? Quel est le résultat par rapport à (3)?

Pour intégrer l'information du réalisateur de ces films, nous pouvons ajoutez des noeuds reprensentant les deux réalisateurs et les relier aux films qu'ils ont réalisés.